In [1]:
import numpy as np
import pandas as pd
import pyodbc
import pyreadr

In [30]:
years = list(range(89, 99))+ [99] + list(range(1400, 1403))
years = [1403]
geo = pd.read_pickle(r"C:\Users\Mahsa\Desktop\geo\processed\geo.pkl")
geo_province = pd.read_pickle(r"C:\Users\Mahsa\Desktop\geo\processed\geo_province.pkl")
geo_province = geo_province.groupby("province_code").agg(
    address = ("province_code", "first"),
    province = ("province", "first")
).reset_index(drop = True)
for year in years:
    # Path to your .accdb file
    file_path = fr'C:\Users\Mahsa\Desktop\geo\raw_data\{year}\Outfile-{year}.accdb'
    # Connection string (for 64-bit Access Database Engine)
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        f'DBQ={file_path};'
    )
    # Connect
    conn = pyodbc.connect(conn_str)
    # List all table names
    tables = [table.table_name for table in conn.cursor().tables(tableType='TABLE')]
    print(year)
    #---------------------------------------------------------------------------------
    Rdata = pd.read_sql(f"SELECT * FROM R{year}Data", conn)
    Udata = pd.read_sql(f"select * from U{year}Data", conn)
    
    if year < 96:
        Rdata = Rdata.loc[:,["Address","MahMorajeh","ShoghlSarparast","NoeKhn"]]
        Rdata["region"] = "Rural"
        Udata = Udata.loc[:,["Address","MahMorajeh","ShoghlSarparast","NoeKhn"]]
        Udata["region"] = "Urban"
        data = pd.concat([Rdata, Udata], axis=0)
        data.rename(columns={"Address":"HHID",
                        "MahMorajeh":"month",
                        "ShoghlSarparast":"head_job",
                        "NoeKhn":"household_type"}, 
                    inplace= True)
        data["head_job"] = data["head_job"].str.strip()
        data.loc[data["month"].notna(),"month"] = data.loc[data["month"].notna(), "month"].astype("int")
        data["quarter"] = ((data["month"]-1)//3)+1
        data["weight"] = np.nan
    elif year >= 96 and year <1400:
        Rdata = Rdata.loc[:,["Address","MahMorajeh","Fasl","weight","NoeKhn"]]
        Rdata["region"] = "Rural"
        Udata = Udata.loc[:,["Address","MahMorajeh","Fasl","weight","NoeKhn"]]
        Udata["region"] = "Urban"
        data = pd.concat([Rdata, Udata], axis=0)
        data.rename(columns={"Address":"HHID",
                        "MahMorajeh":"month",
                        "Fasl": "quarter",
                        "NoeKhn":"household_type"}, 
                    inplace= True)
    elif (year >= 1400 and year <1402) or year==1403:
        Rdata = Rdata.loc[:,["Address","Fasl","weight","NoeKhn"]]
        Rdata["region"] = "Rural"
        Udata = Udata.loc[:,["Address","Fasl","weight","NoeKhn"]]
        Udata["region"] = "Urban"
        data = pd.concat([Rdata, Udata], axis=0)
        data.rename(columns={"Address":"HHID",
                        "Fasl": "quarter",
                        "NoeKhn":"household_type"}, 
                    inplace= True)
        data["month"] = np.nan
    else:
        Rdata = Rdata.loc[:,["Address","Fasl","Weight","NoeKhn"]]
        Rdata["region"] = "Rural"
        Udata = Udata.loc[:,["Address","Fasl","Weight","NoeKhn"]]
        Udata["region"] = "Urban"
        data = pd.concat([Rdata, Udata], axis=0)
        data.rename(columns={"Address":"HHID",
                        "Fasl": "quarter",
                        "NoeKhn":"household_type",
                        "Weight":"weight"}, 
                    inplace= True)
        data["month"] = np.nan
        
    data["HHID"] = data["HHID"].astype("str")
    mask = data["HHID"].str.contains(r"\.")
    data.loc[mask, 'HHID'] = data.loc[mask, "HHID"].str.split(".").str[0]
    data = data.loc[data["HHID"].notna()]
    data.loc[data["month"].notna(),"month"] = data.loc[data["month"].notna(), "month"].astype("int")
    data["month"] = np.where(data["month"] == 1, 12, data["month"]-1)
    data["month"] = np.where(data["month"]>0, data["month"], np.nan)
    if data["household_type"].dtype == object:
        data["household_type"] = data["household_type"].str.strip()  
        data["household_type"] = np.where(data["household_type"]=='1',"NormalResiding",np.where(
            data["household_type"]=='2',"InGroupResiding", np.NaN
        ))
    else:
        data["household_type"] = np.where(data["household_type"]==1,"NormalResiding",np.where(
            data["household_type"]==2,"InGroupResiding", np.NaN
        ))
    data.loc[data["quarter"].notna(),"quarter"] = data.loc[data["quarter"].notna(), "quarter"].astype("int")
    data["quarter"] = np.where(data["quarter"]==1, "spring",np.where(
        data["quarter"]==2,"Summer",np.where(
            data["quarter"]==3, "Fall", np.where(
                data["quarter"]==4, "Winter", np.nan
            )
        )
    ))
    #---------------------------------------------------------------------------------------
    if year <92:
        result = pyreadr.read_r(fr"C:\Users\Mahsa\Desktop\geo\CountyCodes8791\Y{year}ShCode.rda")
        shcode = list(result.values())[0]
        shcode["HHID"] = shcode["HHID"].astype("str")
        mask = shcode["HHID"].str.contains(r"\.")
        shcode.loc[mask, 'HHID'] = shcode.loc[mask, "HHID"].str.split(".").str[0]
        shcode["SHCode"] = shcode["SHCode"].astype("str")
        shcode = shcode[["HHID","SHCode"]]
        data = pd.merge(data, shcode, on="HHID", how="left")
        data.rename(columns={"SHCode":"county_code"}, inplace=True)
        print(
        f"Ministry's file has {shcode.shape[0]} rows\n",
        f"The original data has {data.shape[0]}, rows\n",
        "The ministry file has also used Sum_R89New and Sum_U89New for 1389\n",
        "In 1390 we have a row with HHID missing value\n",
        f"The data for {year} has {data["county_code"].isna().sum()} missing values for county_code\n",
        f"The difference between ministry and original file is exactly equal to \nthe number of missing values : \n{data.shape[0] - shcode.shape[0]} - {data["county_code"].isna().sum()} {data.shape[0] - shcode.shape[0]==data["county_code"].isna().sum()}"
        )
    else:
        data["county_code"] = data["HHID"].str[1:5]
    data["province_code"] = data["HHID"].str[1:3]
    print(
        f"The number of provinces is: {data["province_code"].nunique()}\n",
        f"The number of counties is: {data["county_code"].nunique()}\n",
    )
    #-----------------------------------------------------------------------------------------
    if year <91:
        data.loc[data["county_code"]=='2305',"county_code"]= '3001'
        data.loc[data["county_code"]=='2308',"county_code"]= '3002'
        data.loc[data["county_code"]=='2315',"county_code"]= '3003'
        data.loc[data["county_code"].isin(["3001","3002","3003"]), "province_code"] = '30'
        data = pd.merge(data, geo, left_on="county_code", right_on="address", how= "left")
        data.drop([col for col in list(geo.columns) if col != "county89"], axis = 1, inplace=True)
        print(f"The exact number of counties in {year} is: {geo["county89"].notna().sum()}")
    elif year== 91:
        data.loc[data["county_code"]=='2305',"county_code"]= '3001'
        data.loc[data["county_code"]=='2308',"county_code"]= '3002'
        data.loc[data["county_code"]=='2315',"county_code"]= '3003'
        data.loc[data["county_code"]=='2110',"county_code"]= '2911'
        data.loc[data["county_code"].isin(["3001","3002","3003"]), "province_code"] = '30'
        data.loc[data["county_code"]== '2911', "province_code"] = '29'
        data = pd.merge(data, geo, left_on="county_code", right_on="address", how= "left")
        data.drop([col for col in list(geo.columns) if col != "county91"], axis = 1, inplace=True)
        print(f"The exact number of counties in {year} is: {geo["county91"].notna().sum()}")
    elif year >91 and year<97:
        data = pd.merge(data, geo, left_on="county_code", right_on="address", how= "left")
        data.drop([col for col in list(geo.columns) if col != "county92"], axis = 1, inplace=True)
        print(f"The exact number of counties in {year} is: {geo["county92"].notna().sum()}")
    else:
        data = pd.merge(data, geo, left_on="county_code", right_on="address", how= "left")
        data.drop([col for col in list(geo.columns) if col != f"county{year}"], axis = 1, inplace=True)
        print(f"The exact number of counties in {year} is: {geo[f'county{year}'].notna().sum()}")
    data = pd.merge(data, geo_province, left_on="province_code", right_on="address", how="left")
    data.drop("address",axis = 1, inplace=True)
    print(data.shape)
    print(data.apply(lambda col: col.unique()))
    data.to_pickle(fr"C:\Users\Mahsa\Desktop\geo\base1\base{year}")

1403


C:\Users\Mahsa\AppData\Local\Temp\ipykernel_1952\276938119.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Rdata = pd.read_sql(f"SELECT * FROM R{year}Data", conn)
C:\Users\Mahsa\AppData\Local\Temp\ipykernel_1952\276938119.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Udata = pd.read_sql(f"select * from U{year}Data", conn)


The number of provinces is: 31
 The number of counties is: 428

The exact number of counties in 1403 is: 484
(37505, 10)
HHID              [20102398853, 22405644553, 22704667446, 214065...
quarter                              [spring, Summer, Fall, Winter]
weight            [718, 240, 256, 149, 406, 206, 493, 145, 451, ...
household_type                    [NormalResiding, InGroupResiding]
region                                               [Rural, Urban]
month                                                         [nan]
county_code       [0102, 2405, 2704, 1406, 0922, 2802, 0608, 290...
province_code     [01, 24, 27, 14, 09, 28, 06, 29, 18, 26, 10, 0...
county1403        [آستانه اشرفیه, گرمی, كردكوی, کوهرنگ, فریمان, ...
province          [گیلان, اردبیل, گلستان, چهارمحال و بختیاری, خر...
dtype: object


In [5]:
years = list(range(89, 99))+ [99] + list(range(1400, 1402))
for year in years:
    mydf = pd.read_pickle(fr"C:\Users\Mahsa\Desktop\geo\base1\base{year}")
    print(f"\n--------------------{year}-----------------\n")
    print(mydf.shape[0])
    print(mydf.isna().sum())
    if year <1400:
        theirdf = pyreadr.read_r(fr"C:\Users\Mahsa\Desktop\base\energy poverty\poverty\HEIS\DataProcessed\Y{year}HHBase.rda")
    else:
        year = year - 1300
        theirdf = pyreadr.read_r(fr"C:\Users\Mahsa\Desktop\base\energy poverty\poverty\HEIS\DataProcessed\Y{year}HHBase.rda")
    theirdf = list(theirdf.values())[0]
    print(theirdf.shape[0])
    print(f"\n{mydf.shape[0] - theirdf.shape[0]}\n\n\n\n")
    


--------------------89-----------------

38950
HHID                0
month               0
head_job            0
household_type      0
region              0
quarter             0
county_code       665
province_code       0
county89          665
province            0
dtype: int64
38285

665





--------------------90-----------------

40011
HHID                 0
month                1
head_job             4
household_type       0
region               0
quarter              0
county_code       1498
province_code        0
county89          1498
province             1
dtype: int64
38513

1498





--------------------91-----------------

40007
HHID                 0
month                0
head_job             0
household_type       0
region               0
quarter              0
county_code       1815
province_code        0
county91          1815
province             0
dtype: int64
38192

1815





--------------------92-----------------

39864
HHID                 0
month              